In [1]:
# import everything and define a test runner function
from importlib import reload
from helper import run_test

import block
import ecc
import helper
import script
import tx

### Exercise 1

#### 1.1. Make [this test](/edit/session6/tx.py) pass
```
tx.py:TxTest:test_is_coinbase
```

In [2]:
# Exercise 1.1

reload(tx)
run_test(tx.TxTest('test_is_coinbase'))

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


### Exercise 2

#### 2.1. Make [this test](/edit/session6/tx.py) pass
```
tx.py:TxTest:test_coinbase_height
```

In [3]:
# Exercise 2.1

reload(tx)
run_test(tx.TxTest('test_coinbase_height'))

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


### Exercise 3

#### 3.1. Find the output address corresponding to this ScriptPubKey
```
76a914338c84849423992471bffb1a54a8d9b1d69dc28a88ac
```

Remember the structure of pay-to-pubkey-hash (p2pkh) which has `OP_DUP OP_HASH160 <hash> OP_EQUALVERIFY OP_CHECKSIG`.

You need to grab the hash160 and turn that into an address.

#### 3.2. Make [this test](/edit/session6/script.py) pass
```
script.py:ScriptTest:test_address
```

In [4]:
# Exercise 3.1

from helper import h160_to_p2pkh_address
from script import Script

hex_script_pubkey = '76a914338c84849423992471bffb1a54a8d9b1d69dc28a88ac'

# bytes.fromhex to get binary
script_pubkey = bytes.fromhex(hex_script_pubkey)
# parse with Script
s = Script.parse(script_pubkey)
# get the 3rd element, which should be the hash160
h160 = s.elements[2]
# convert h160 to p2pkh address
print(h160_to_p2pkh_address(h160))

15hZo812Lx266Dot6T52krxpnhrNiaqHya


In [5]:
# Exercise 3.2

reload(script)
run_test(script.ScriptTest('test_address'))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


### Exercise 4

#### 4.1. What is the double_sha256 of this block? Notice anything?
```
020000208ec39428b17323fa0ddec8e887b4a7c53b8c0a0a220cfd0000000000000000005b0750fce0a889502d40508d39576821155e9c9e3f5c3157f961db38fd8b25be1e77a759e93c0118a4ffd71d
```

#### 4.2. Make [these tests](/edit/session6/block.py) pass.
```
block.py:BlockTest:test_parse
block.py:BlockTest:test_serialize
block.py:BlockTest:test_hash
```

In [6]:
# Exercise 4.1

from helper import double_sha256

hex_block = '020000208ec39428b17323fa0ddec8e887b4a7c53b8c0a0a220cfd0000000000000000005b0750fce0a889502d40508d39576821155e9c9e3f5c3157f961db38fd8b25be1e77a759e93c0118a4ffd71d'

# bytes.fromhex to get the binary
bin_block = bytes.fromhex(hex_block)
# double_sha256 the result
result = double_sha256(bin_block)
# hex() to see what it looks like
print(result.hex())

2375044d646ad73594dd0b37b113becdb03964584c9e7e000000000000000000


In [7]:
# Exercise 4.2

reload(block)
run_test(block.BlockTest('test_parse'))
run_test(block.BlockTest('test_serialize'))
run_test(block.BlockTest('test_hash'))

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK
.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK
.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


In [8]:
# Version Signaling Example

from io import BytesIO
from block import Block

hex_block = '020000208ec39428b17323fa0ddec8e887b4a7c53b8c0a0a220cfd0000000000000000005b0750fce0a889502d40508d39576821155e9c9e3f5c3157f961db38fd8b25be1e77a759e93c0118a4ffd71d'

# bytes.fromhex to get the binary block
bin_block = bytes.fromhex(hex_block)
# create a stream using BytesIO
stream = BytesIO(bin_block)
# parse the block
b = Block.parse(stream)
# get the version
version = b.version
# rightshift 29 (version >> 29) and see if it's equal to 0b001 for BIP9
print('BIP9: {}'.format(version >> 29 == 0b001))
# see if bit 4 (version >> 4) from the right is set ( & 1) for BIP91
print('BIP91: {}'.format(version >> 4 & 1 == 1))
# see if bit 1 (version >> 1) from the right is set ( & 1) for BIP141
print('BIP141: {}'.format(version >> 1 & 1 == 1))

BIP9: True
BIP91: False
BIP141: True


### Exercise 5

#### 5.1. Make [these tests](/edit/session6/block.py) pass.
```
block.py:BlockTest:test_bip9
block.py:BlockTest:test_bip91
block.py:BlockTest:test_bip141
```

In [9]:
# Exercise 5.1

reload(block)
run_test(block.BlockTest('test_bip9'))
run_test(block.BlockTest('test_bip91'))
run_test(block.BlockTest('test_bip141'))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK
.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK
.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


In [10]:
# Calculating Target from Bits Example

from helper import little_endian_to_int

bits = bytes.fromhex('e93c0118')
exponent = bits[-1]
coefficient = little_endian_to_int(bits[:-1])
target = coefficient*2**(8*(exponent-3))
print('{:x}'.format(target).zfill(64))

0000000000000000013ce9000000000000000000000000000000000000000000


In [11]:
# Calculating Difficulty from Target Example

from helper import little_endian_to_int

bits = bytes.fromhex('e93c0118')
exponent = bits[-1]
coefficient = little_endian_to_int(bits[:-1])
target = coefficient*2**(8*(exponent-3))

bits_min = bytes.fromhex('ffff001d')
exponent_min = bits_min[-1]
coefficient_min = little_endian_to_int(bits_min[:-1])
min = coefficient_min*2**(8*(exponent_min-3))

difficulty = min // target
print(difficulty)

888171856257


### Exercise 6

#### 6.1. Calculate the target and difficulty for these bits:
```
f2881718
```

Bits to target formula is 

\\(coefficient\cdot2^{8\cdot(exponent-3)}\\) 

where coefficient is the first three bytes in little endian and exponent is the last byte.

Target to Difficulty formula is 

\\(difficulty = min / target\\)

where \\(min = 0xffff\cdot2^{8\cdot(0x1d-3)}\\)

#### 6.2. Make [this test](/edit/session6/block.py) pass.
```
block.py:BlockTest:test_target
```

In [12]:
# Exercise 6.1

from helper import little_endian_to_int

hex_bits = 'f2881718'

# bytes.fromhex to get the bits
bits = bytes.fromhex(hex_bits)
# last byte is exponent
exponent = bits[-1]
# first three bytes are the coefficient in little endian
coefficient = little_endian_to_int(bits[:-1])
# plug into formula coefficient * 2^(8*(exponent-3)) to get the target
target = coefficient * 2**(8*(exponent-3))
# print target using print('{:x}'.format(target).zfill(64))
print('{:x}'.format(target).zfill(64))

hex_min_bits = 'ffff001d'
# do the same for the min_bits for min target
min_bits = bytes.fromhex(hex_min_bits)
min_exponent = min_bits[-1]
min_coefficient = little_endian_to_int(min_bits[:-1])
min_target = min_coefficient * 2**(8*(min_exponent-3))

# difficulty is min target / target
difficulty = min_target / target
print(difficulty)

00000000000000001788f2000000000000000000000000000000000000000000
46717549644.70642


In [13]:
# Exercise 6.2

reload(block)
run_test(block.BlockTest('test_target'))

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


### Exercise 7

#### 7.1.Validate the proof-of-work for this block
```
04000000fbedbbf0cfdaf278c094f187f2eb987c86a199da22bbb20400000000000000007b7697b29129648fa08b4bcd13c9d5e60abb973a1efac9c8d573c71c807c56c3d6213557faa80518c3737ec1
```

Check that the proof-of-work (double-sha256 interpreted as a little-endian number) is lower than the target.

#### 7.2. Make [this test](/edit/session6/block.py) pass.
```
block.py:BlockTest:test_check_pow
```

In [14]:
# Exercise 7.1

from io import BytesIO
from helper import double_sha256, little_endian_to_int
from block import Block

hex_block = '04000000fbedbbf0cfdaf278c094f187f2eb987c86a199da22bbb20400000000000000007b7697b29129648fa08b4bcd13c9d5e60abb973a1efac9c8d573c71c807c56c3d6213557faa80518c3737ec1'

# bytes.fromhex to get the binary block
bin_block = bytes.fromhex(hex_block)
# double_sha256 the block
sha = double_sha256(bin_block)
# interpret the result as a number in little endian
proof = little_endian_to_int(sha)
# make a stream using BytesIO
stream = BytesIO(bin_block)
# parse the Block
b = Block.parse(stream)
# get the target
target = b.target()
# check proof of work < target
print(proof < target)

True


In [15]:
# Exercise 7.2

reload(block)
run_test(block.BlockTest('test_check_pow'))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK
